In [1]:
import stk, stko

import py3Dmol

from rdkit.Chem import AllChem

In [2]:
def show_stk_mol(stk_mol):
    
    data = AllChem.MolToMolBlock(stk_mol.to_rdkit_mol())
    
    p = py3Dmol.view(
        
        data=data,
        
        style={'stick':{'colorscheme':'greyCarbon'}}, 
        
        width=400,
        
        height=400,
        
    )
    p.setBackgroundColor('0xeeeeee')
    
    p.zoomTo()
    
    p.show()

In [3]:
smiles = 'O=C(O)c1ccccc1O'

In [4]:
acid = stk.BuildingBlock( smiles, [stk.CarboxylicAcidFactory()] )

show_stk_mol( acid )

stk.XyzWriter().write( acid, './acid.xyz' )

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

In [6]:
from ase.io import read, write

atoms = read( './acid.xyz' )

from tblite.ase import TBLite

atoms.calc = TBLite(method="GFN1-xTB")


from ase.optimize import BFGS

dynamics = BFGS( atoms, trajectory = 'trajectory.traj' )

dynamics.run( fmax=0.05 )

ModuleNotFoundError: No module named 'tblite'

In [6]:
class Atom(dict):
    
    def __init__(self, line):
        
        self["type"] = line[0:6].strip()
        
        self["idx"] = line[6:11].strip()
        
        self["name"] = line[12:16].strip()
        
        self["resname"] = line[17:20].strip()
        
        self["resid"] = int(int(line[22:26]))
        
        self["x"] = float(line[30:38])
        
        self["y"] = float(line[38:46])
        
        self["z"] = float(line[46:54])
        
        self["sym"] = line[76:78].strip()

    def __str__(self):
        
        line = list(" " * 80)

        line[0:6] = self["type"].ljust(6)
        
        line[6:11] = self["idx"].ljust(5)
        
        line[12:16] = self["name"].ljust(4)
        
        line[17:20] = self["resname"].ljust(3)
        
        line[22:26] = str(self["resid"]).ljust(4)
        
        line[30:38] = str(self["x"]).rjust(8)
        
        line[38:46] = str(self["y"]).rjust(8)
        
        line[46:54] = str(self["z"]).rjust(8)
        
        line[76:78] = self["sym"].rjust(2)
        
        return "".join(line) + "\n"


In [7]:
class Molecule(list):
    
    def __init__(self, file):
        
        for line in file:
            
            if "ATOM" in line or "HETATM" in line:
                
                self.append(Atom(line))

    def __str__(self):
        
        outstr = ""
        
        for at in self:
            
            outstr += str(at)

        return outstr


In [8]:
from ase.io.trajectory import Trajectory

trajectory = Trajectory('./trajectory.traj')

view = py3Dmol.view(width=400, height=300)

molecules = []

for atoms in trajectory:

    write( 'intermediary.pdb', atoms )

    with open( 'intermediary.pdb' ) as ifile: molecule = Molecule(ifile)

    molecules.append( molecule )

In [9]:
for mol in molecules:
    
    for at in mol:
        
        if at["resname"] == "PRO":
            
            at["pymol"] = {"stick": {'color': "yellow"}}
            
        elif at["resname"] == "GLY":
            
            at["pymol"] = {"stick": {'color': 'blue'}}

In [10]:
view = py3Dmol.view(width=400, height=300)

models = ""

for i, mol in enumerate(molecules):
    
    models += "MODEL " + str(i) + "\n"
    
    models += str(mol)
    
    models += "ENDMDL\n"
    
view.addModelsAsFrames(models)

for i, at in enumerate(molecules[0]):
    
    default = {'stick':{'colorscheme':'greyCarbon'}}
    
    view.setStyle({'model': -1, 'serial': i+1}, at.get("pymol", default))

view.zoomTo()

view.animate({'loop': "forward"})

view.show()


You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol